# Snowflake + Dask

How to load data from a Snowflake table or query into a Dask dataframe

## Connect to Snowflake

See [README](README.md) for more details on how to set up the credentials file.

In [1]:
import os
import snowflake.connector

In [46]:
conn_info = {
    "warehouse" : "COMPUTE_WH",
    "database": "SNOWFLAKE_SAMPLE_DATA",
    "schema": "TPCDS_SF100TCL",
    "account": "mf80263.us-east-2.aws",
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_PASSWORD"]
}
conn = snowflake.connector.connect(**conn_info)

## Setup query

We need to set up a query that will return chunks of the full result based on a column in the data. These become our partitions in a Dask dataframe. We use a [binding for the Snowflake query](https://docs.snowflake.com/en/user-guide/python-connector-example.html#binding-data) so that we can pass different values at execution time.

In [9]:
query = """
    SELECT
        *
    FROM
        customer
    LIMIT 10
"""

In [10]:
help(conn.cursor().execute)

Help on method execute in module snowflake.connector.cursor:

execute(command, params=None, timeout=None, _do_reset=True, _put_callback=None, _put_azure_callback=None, _put_callback_output_stream=<ipykernel.iostream.OutStream object at 0x7f5f74b8a290>, _get_callback=None, _get_azure_callback=None, _get_callback_output_stream=<ipykernel.iostream.OutStream object at 0x7f5f74b8a290>, _show_progress_bar=True, _statement_params=None, _is_internal=False, _no_results=False, _use_ijson=False, _is_put_get=None, _raise_put_get_error=True, _force_put_overwrite=False) method of snowflake.connector.cursor.SnowflakeCursor instance
    Executes a command/query



Validate the query is good with pandas

In [11]:
cur = conn.cursor()
df = cur.execute(command=query).fetch_pandas_all()
len(df), df.memory_usage().sum() / 1e6  # memory size in MB

(10, 0.001108)

In [12]:
df

,C_CUSTOMER_SK,C_CUSTOMER_ID,C_CURRENT_CDEMO_SK,C_CURRENT_HDEMO_SK,C_CURRENT_ADDR_SK,C_FIRST_SHIPTO_DATE_SK,C_FIRST_SALES_DATE_SK,C_SALUTATION,C_FIRST_NAME,C_LAST_NAME,C_PREFERRED_CUST_FLAG,C_BIRTH_DAY,C_BIRTH_MONTH,C_BIRTH_YEAR,C_BIRTH_COUNTRY,C_LOGIN,C_EMAIL_ADDRESS,C_LAST_REVIEW_DATE
0,94911797,AAAAAAAAFDNDIKFA,1604095,1444,4537993,2449587,2449557,Mrs.,Annie,Crawley,N,9,6,1953,NAURU,None,Annie.Crawley@T05PnHEpPs572Sp.org,2452506
1,94911798,AAAAAAAAGDNDIKFA,427153,5748,36805143,2449682,2449652,Mr.,Troy,Poe,N,14,6,1989,BRAZIL,None,Troy.Poe@F58FuaxTaU72Sy.edu,2452394
2,94911799,AAAAAAAAHDNDIKFA,68714,5386,26519305,2449270,2449240,Sir,Mark,Baker,Y,11,3,1983,BURKINA FASO,None,Mark.Baker@mF9DCZVtbVPcPuz.edu,2452532
3,94911800,AAAAAAAAIDNDIKFA,303248,3046,36401244,2449277,2449247,Sir,Robert,Parker,Y,2,6,1926,KIRIBATI,None,Robert.Parker@tm2MJA5YZsSoeDZqh.edu,2452513
4,94911801,AAAAAAAAJDNDIKFA,1404903,5226,6517996,2450908,2450878,Ms.,Diana,Adams,Y,22,5,1932,HUNGARY,None,Diana.Adams@2n.edu,2452360
5,94911802,AAAAAAAAKDNDIKFA,1560715,820,15056125,2450948,2450918,Sir,Robert,Jones,Y,1,10,1967,ZIMBABWE,None,Robert.Jones@dHeuQQZTqg.org,2452564
6,94911803,AAAAAAAALDNDIKFA,1886448,3975,17879902,2451877,2451847,Mrs.,Jennifer,Kelly,N,7,8,1985,CHILE,None,Jennifer.Kelly@6aSquozg.edu,2452421
7,94911804,AAAAAAAAMDNDIKFA,372497,124,40856551,2451566,2451536,Dr.,Anna,Woo,Y,27,6,1927,DJIBOUTI,None,Anna.Woo@e1.org,2452390
8,94911805,AAAAAAAANDNDIKFA,490712,73,12339834,2451578,2451548,Dr.,Laura,Dodge,Y,14,5,1982,LIBERIA,None,Laura.Dodge@1mfoQ8pX.edu,2452444
9,94911806,AAAAAAAAODNDIKFA,630274,7175,25923276,2449958,2449928,Ms.,Lisa,Morris,Y,15,8,1953,RWANDA,None,Lisa.Morris@ghpoc7XulHz1mN4.com,2452352


## Initialize Dask cluster

In [13]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster
import time

n_workers = 4
cluster = SaturnCluster(
    n_workers=n_workers,
    scheduler_size='medium',
    worker_size='xlarge',
    nthreads=2
)
client = Client(cluster)
cluster

[2020-09-04 19:57:05] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 19:57:12] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 19:57:31] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 19:57:52] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 19:58:26] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 19:59:20] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 20:00:04] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 20:00:35] INFO - dask-saturn | Starting cluster. Status: pending
[2020-09-04 20:01:30] INFO - dask-saturn | Cluster is ready


If you initialized your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready

> **Pro tip:** Create and/or start your cluster from the "Dask" page in Saturn if you want to get a head start!

In [14]:
while len(client.scheduler_info()['workers']) < n_workers:
    print('Waiting for workers, got', len(client.scheduler_info()['workers']))
    time.sleep(30)
print('Done!')

Waiting for workers, got 0
Done!


In [15]:
import dask.dataframe as dd

## Load larger data with Dask!

We set up a function with `dask.delayed`. `@delayed` is a decorator that turns a Python function into a function suitable for running on the Dask cluster. When you execute a delayed function, instead of executing the operation, it returns a delayed result that represents what the return value of the function will be. `dask.dataframe.from_delayed` takes a list of these delayed objects, and concatenates them into a Dask dataframe.

In [39]:
query = """
    SELECT
        *
    FROM
        customer
    WHERE
        C_BIRTH_DAY = %s
    LIMIT 100
    """

In [79]:
cur = conn.cursor()
x = cur.execute(query, 1)

In [85]:
# relevant files
# * https://github.com/snowflakedb/snowflake-connector-python/blob/c058e205a34c2ce2971769fc790ffe0bce552023/src/snowflake/connector/cursor.py
# * https://github.com/snowflakedb/snowflake-connector-python/blob/0cf126e0791787d9841bb4849feb781597e8671f/src/snowflake/connector/chunk_downloader.py
# * https://github.com/snowflakedb/snowflake-connector-python/blob/0cf126e0791787d9841bb4849feb781597e8671f/src/snowflake/connector/arrow_result.pyx

In [84]:
x._result._fetch_pandas_all()

,C_BIRTH_DAY
0,1
1,1
2,1
3,1
4,1
...,...
95,1
96,1
97,1
98,1


In [77]:
cur.check_can_use_arrow_resultset()

In [47]:
from dask import delayed
import dask.dataframe as dd

@delayed
def get_chunk(conn_info, query, day_of_month):
    conn = snowflake.connector.connect(**conn_info)
    cur = conn.cursor().execute(query, day_of_month)
    return cur.fetch_pandas_all()

Notice that now `ddf` has 3 partitions, corresponding to the 3 days we chose to load

In [59]:
from dask import delayed
import dask.dataframe as dd

query = """
    SELECT
        C_BIRTH_DAY
    FROM
        customer
    WHERE
        C_BIRTH_DAY = %s
    LIMIT 100
    """

@delayed
def get_chunk(conn_info, query, day_of_month):
    conn = snowflake.connector.connect(**conn_info)
    cur = conn.cursor().execute(query, day_of_month)
    return cur.fetch_pandas_all()

ddf = dd.from_delayed(
    [
        get_chunk(conn_info, query, day_of_month)
        for day_of_month in range(10, 15)
    ],
)
ddf

,C_BIRTH_DAY
npartitions=5,
,int8
,...
...,...
,...
,...


<br>
The cell below will execute the Snowflake queries across the cluster and compute the length and size of each partition

In [60]:
len(ddf), ddf.memory_usage().sum().compute() / 1e6

(500, 0.00114)

The partitions in the Dask dataframe are pandas dataframes

In [ ]:
ddf_part = ddf.partitions[0].compute()
type(ddf_part)

If we are performing a lot of operations using this Dask dataframe (such as training a machine learning model), and the data will fit in the memory of the _cluster_, we should `persist()` the dataframe to perform all the loading up-front.

In [ ]:
from dask.distributed import wait

ddf = ddf.persist()
_ = wait(ddf)